### Interactive Time Series with Python and API: Generating Heatmap of Taxis in Singapore
https://towardsdatascience.com/interactive-time-series-with-python-and-api-generating-heatmap-of-taxis-in-singapore-4d84adbd4c54

In [1]:
import folium
import folium.plugins as plugins
import pandas as pd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import datetime

In [2]:
cumulative = pd.DataFrame()

In [3]:
mins = input('How many mins do you want to see? ')
number_of_times = (int(mins)*60)/30

In [4]:
startTime = time.time()
for i in range(int(number_of_times)):
    url = ("https://api.data.gov.sg/v1/transport/taxi-availability")
    response = requests.get(url)
    data = response.json()
    df = pd.io.json.json_normalize(data['features'])
    coordinateslist = df['geometry.coordinates'].tolist()
    df1 = pd.DataFrame(coordinateslist)
    result = df1.transpose()
    result.columns = ['coordinates']
    result['Timestamp'] = (df['properties.timestamp'][0])
    cumulative = cumulative.append(result) 
    time.sleep(30)
endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime,'seconds')

C:\Users\Youss\AppData\Local\Temp\ipykernel_7984\740301445.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = pd.io.json.json_normalize(data['features'])
C:\Users\Youss\AppData\Local\Temp\ipykernel_7984\740301445.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative = cumulative.append(result)
C:\Users\Youss\AppData\Local\Temp\ipykernel_7984\740301445.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = pd.io.json.json_normalize(data['features'])
C:\Users\Youss\AppData\Local\Temp\ipykernel_7984\740301445.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cumulative = cumulative.append(result)
C:\Users\Youss\AppData\Local\Temp\ipykernel_7984\740301445.py:6: FutureWarning: pandas.io.json.jso

Elapsed Time = 124.53110265731812 seconds


In [5]:
cumulative['coordinates'] = cumulative['coordinates'].astype(str)
Latitude = []
Longitude = []
coordinates = []
for i in cumulative['coordinates']:
    i = i.split(", ")
    lat = i[1][:-1]
    long = i[0][1:]
    Latitude.append(lat)
    Longitude.append(long)
cumulative['Latitude'] = Latitude
cumulative['Longitude'] = Longitude

In [6]:
lat_long_list = []
for i in cumulative['Timestamp'].unique():
    temp=[]
    for index, instance in cumulative[cumulative['Timestamp'] == i].iterrows():
        temp.append([instance['Latitude'],instance['Longitude']])
    lat_long_list.append(temp)

In [7]:
#converting it to datetime format
cumulative['Timestamp']= pd.to_datetime(cumulative['Timestamp'])#creating a time index
time_index = []
for i in cumulative['Timestamp'].unique():
    time_index.append(i)#formatting the index
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [10]:
#Choosing the map type 
m = folium.Map(location=[1.352083,103.819839],zoom_start = 11, tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",attr="Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community")#Plot it on the map
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster",index=date_strings,max_opacity=0.7).add_to(m)# Display the map
m